# <center> **Recurrent Neural Network(RNNs)** </center>

--> ANNs to handle sequential data

You need to develop RNN models that can do the following:


*   Handle variable-length input sequences

*   Track long-term dependencies in the data

*   Maintain information about the sequence's order

*   Share parameters across the entirety of the sequence

### **Training an ANN for Sequential Data - Nvidia Stock prediction**


*   **target variable** -> Price of stock in a given day
*   **features** -> Price of a stock in a previous 60 days

In [1]:
# import required libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

1. Import Nvidia Stock Data:

In [3]:
# import dataset
data = pd.read_csv("https://raw.githubusercontent.com/PacktWorkshops/The-TensorFlow-Workshop/master/Chapter09/Datasets/NVDA.csv")

# display first 5 rows of the dataset
data.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2015-07-22,19.650000,19.650000,19.17,19.410000,18.851749,8911800
1,2015-07-23,19.450001,19.940001,19.41,19.650000,19.084845,4247900
2,2015-07-24,19.790001,19.809999,19.34,19.420000,18.861464,4721100
3,2015-07-27,19.250000,19.530001,19.09,19.309999,18.754622,4810500
4,2015-07-28,19.360001,19.860001,19.16,19.730000,19.162542,4957700


2. Split the dataset into training and test sets:

In [4]:
#  use data before "2019-01-01" as training set
data_training = data[data["Date"] < "2019-01-01"].copy()

# use data after "2019-01-01" as testing set
data_test = data[data["Date"] >= "2019-01-01"].copy()

In [5]:
# drop the un-necessary columns after splitting
training_data = data_training.drop(["Date", "Adj Close"], axis=1)
test_data = data_test.drop(["Date", "Adj Close"], axis=1)

# display the changed dataframe
training_data.head()

,Open,High,Low,Close,Volume
0,19.650000,19.650000,19.17,19.410000,8911800
1,19.450001,19.940001,19.41,19.650000,4247900
2,19.790001,19.809999,19.34,19.420000,4721100
3,19.250000,19.530001,19.09,19.309999,4810500
4,19.360001,19.860001,19.16,19.730000,4957700


3. Rescale the training data in a uniform range

In [6]:
# store the column names before rescaling
train_cols = training_data.columns
test_cols = test_data.columns

train_cols

Index(['Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')

In [7]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# initialize the scaler
scaler = MinMaxScaler()

# perform the rescaling
training_data = scaler.fit_transform(training_data)
# training_data = pd.DataFrame(training_data, columns=train_cols)
training_data.head()

AttributeError: 'numpy.ndarray' object has no attribute 'head'

4. Prepare data from traning and testing

In [8]:
# check the shape of training_data
training_data.shape[0]

868

In [9]:
# split your data into X_train and y_train
X_train = []
y_train = []

# Looping through the range of 60 to the number of rows in training_data
for i in range(60, training_data.shape[0]):

    # Appending the values of training_data from i-60 to i to X_train
    X_train.append(training_data[i-60:i])

    # Appending the value of training_data at index i, column 0 to y_train
    y_train.append(training_data[i, 0])

In [10]:
# convert X_train and y_train into NumPy arrays
X_train, y_train = np.array(X_train), np.array(y_train)

# display the dimension of X_train, y_train
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)

X_train: (808, 60, 5)
y_train: (808,)


--> As shown above, the prepared training set contains 808 observations with 60 days of data for the five features.

##### Transform the data into a 2D matrix with the shape of the sample (the number of samples and the number of features in each sample).<br>
##### Stack the features for all 60 days on top of each other to get an output size of (808, 300)

In [12]:
# save the shape of X_train
X_old_shape = X_train.shape

# reshaping X_train to have X_old_shape[0] rows and X_old_shape[1]*X_old_shape[2] columns
X_train = X_train.reshape(X_old_shape[0], X_old_shape[1]*X_old_shape[2])

# display the reshaped X_train
X_train.shape

(808, 300)

5. Build an ANN model.

In [13]:
# import libraries
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout

In [14]:
# initialize the ANN model
regressor_ann = Sequential()

In [16]:
# add an input layer with input_shape as 300
regressor_ann.add(Input(shape = (X_train.shape[1])))

# add dense layer with 512 units as output
regressor_ann.add(Dense(512, activation="relu"))
# add dropout layer to remove 20% of the units during training to prevent overfitting
regressor_ann.add(Dropout(0.2))

# add another dense layer and Dropout layer
regressor_ann.add(Dense(128, activation="relu"))
# add another Dropout layer
regressor_ann.add(Dropout(0.3))

# add another dense and dropout layer
regressor_ann.add(Dense(64, activation="relu"))
regressor_ann.add(Dropout(0.4))

# add another dense and dropout layer
regressor_ann.add(Dense(16, activation="relu"))
regressor_ann.add(Dropout(0.5))

# add final dense layer as output
regressor_ann.add(Dense(1))

In [17]:
# display of the model architecture
regressor_ann.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               154112    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               65664     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 16)                1

6. Compile the model with optimizer, loss function and accuracy metrics.

In [18]:
# compile the model for training
regressor_ann.compile(
    optimizer="adam",
    loss="mean_squared_error",
    metrics="mse"
                    )

7. Train the ANN model

In [20]:
# fit the ANN model to the training data
regressor_ann.fit(X_train,
                  y_train,
                  epochs=20,
                  batch_size=32
                  )

Epoch 1/20
26/26 [==============================] - 0s 2ms/step - loss: 0.0438 - mse: 0.0438
Epoch 2/20
26/26 [==============================] - 0s 2ms/step - loss: 0.0410 - mse: 0.0410
Epoch 3/20
26/26 [==============================] - 0s 1ms/step - loss: 0.0396 - mse: 0.0396
Epoch 4/20
26/26 [==============================] - 0s 1ms/step - loss: 0.0395 - mse: 0.0395
Epoch 5/20
26/26 [==============================] - 0s 2ms/step - loss: 0.0402 - mse: 0.0402
Epoch 6/20
26/26 [==============================] - 0s 2ms/step - loss: 0.0326 - mse: 0.0326
Epoch 7/20
26/26 [==============================] - 0s 1ms/step - loss: 0.0335 - mse: 0.0335
Epoch 8/20
26/26 [==============================] - 0s 2ms/step - loss: 0.0297 - mse: 0.0297
Epoch 9/20
26/26 [==============================] - 0s 1ms/step - loss: 0.0311 - mse: 0.0311
Epoch 10/20
26/26 [==============================] - 0s 1ms/step - loss: 0.0304 - mse: 0.0304
Epoch 11/20
26/26 [==============================] - 0s 2ms/step - lo

# <center><b>Recurrent Neural Networks(RNN)</b></center>